## Try to disassemble and recreate the syn6.o binary

Parse out the `syn6.o` binary.

In [50]:
syn6 = open("syn6.o",'rb')
syn6.seek(0)
print("Magic number",syn6.read(2).hex(), "Alcyon-Format Object Module")
text_size = syn6.read(4)
print("Text Size",text_size.hex())
data_size = syn6.read(4)
print("Data Size",data_size.hex())
print("BSS Size",syn6.read(4).hex())
sym_size = syn6.read(4)
print("Symbol Size",sym_size.hex())
print("Reserved (must be zero)",syn6.read(10).hex())
text_data = syn6.read(0x2178)
print("Symbol Table")
sym_size_int = int.from_bytes(sym_size, "big")
for i in range(0,int(sym_size_int/14)):
    print(syn6.read(8), syn6.read(4).hex(),syn6.read(2).hex())
relocation_table = syn6.read()

print("Relocation Fix Up Information")
print(hex(len(relocation_table)))


Magic number 601a Alcyon-Format Object Module
Text Size 00002178
Data Size 00000000
BSS Size 00000000
Symbol Size 00001d34
Reserved (must be zero) 00000000000000000000
Symbol Table
b'DISABLET' 88000000 0000
b'ENABLETI' 88000000 0000
b'INITDSP\x00' a2000000 0026
b'COPYLOOP' 82000000 0056
b'MAKESAMV' 82000000 00b4
b'BG\x00\x00\x00\x00\x00\x00' c00000f0 0058
b'HC\x00\x00\x00\x00\x00\x00' c00000f0 0004
b'HP\x00\x00\x00\x00\x00\x00' c00000f0 002e
b'HS\x00\x00\x00\x00\x00\x00' c00000f0 0034
b'VC\x00\x00\x00\x00\x00\x00' c00000f0 0006
b'VI\x00\x00\x00\x00\x00\x00' c00000f0 004e
b'VP\x00\x00\x00\x00\x00\x00' c00000f0 003e
b'VS\x00\x00\x00\x00\x00\x00' c00000f0 0044
b'HBB\x00\x00\x00\x00\x00' c00000f0 0030
b'HBE\x00\x00\x00\x00\x00' c00000f0 0032
b'HDE\x00\x00\x00\x00\x00' c00000f0 003c
b'HEQ\x00\x00\x00\x00\x00' c00000f0 0054
b'HVS\x00\x00\x00\x00\x00' c00000f0 0036
b'LPH\x00\x00\x00\x00\x00' c00000f0 0008
b'LPV\x00\x00\x00\x00\x00' c00000f0 000a
b'OB0\x00\x00\x00\x00\x00' c00000f0 0010
b'OB1\

In [162]:
0x2178 + 0x1d34

16044

In [161]:
!wc -c syn6.o

24640 syn6.o


In [52]:
open("syn6.bin",'wb').write(text_data)


8568

Now go into IDA Pro, disassemble (using M68000) and export as syn6.s

[do stuff in ida pro]

Now let's rebuild using rmac. Download and build rmac:

In [3]:
import os
!rm -rf rmac
!git clone https://github.com/mwenge/rmac.git
os.chdir('rmac')
!make -j5
os.chdir('..')


Cloning into 'rmac'...
remote: Enumerating objects: 2050, done.
remote: Counting objects: 100% (2050/2050), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 2050 (delta 1641), reused 2050 (delta 1641), pack-reused 0
Receiving objects: 100% (2050/2050), 740.86 KiB | 2.64 MiB/s, done.
Resolving deltas: 100% (1641/1641), done.
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c kwgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c debug.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56k.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c eagen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o dsp56kgen dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o 68kgen 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o kwgen kwgen.c
gcc -std=c99 -D

In file included from macro.h:12,
                 from macro.c:9:
macro.c: In function ‘ExitMacro’:
rmac.h:78:30: note: ‘#pragma message: !!! Bad macro exiting !!!’
   78 |         #define DO_PRAGMA(x) _Pragma (#x)
      |                              ^~~~~~~
rmac.h:79:31: note: in expansion of macro ‘DO_PRAGMA’
   79 |         #define WARNING(desc) DO_PRAGMA(message (#desc))
      |                               ^~~~~~~~~
macro.c:54:1: note: in expansion of macro ‘WARNING’
   54 | WARNING(!!! Bad macro exiting !!!)
      | ^~~~~~~
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c object.c
./kwgen mo <op.tab >opkw.h
./dsp56kgen dsp56k.tab <dsp56k.mch >dsp56ktab.h
./kwgen mr <risc.tab >risckw.h
./kwgen mp <6502.tab >6502kw.h
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c rmac.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c sect.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c symbol.c
./kwgen kw <kw.tab >kwtab.h
./68kgen 68k.tab <68k.mch >6

Rebuild syn6.s using rmac and find the differences:

In [160]:
!./rmac/rmac -s ~oall -n -l*syn6prn -ps -i syn6.s -o syn6
!diff -y -W 120 <(dd if=syn6|xxd) <(dd if=syn6.o|xxd)

Legacy mode OFF
syn6.s 84: Warning: unoptimized short branch
syn6.s 1022: Warning: unoptimized short branch
27+1 records in
27+1 records out
48+1 records in
48+1 records out
24640 bytes (25 kB, 24 KiB) copied, 5.3004e-05 s, 465 MB/s
14279 bytes (14 kB, 14 KiB) copied, 4.7309e-05 s, 302 MB/s
00000000: 601a 0000 2178 0000 0000 0000 0000 0000  `...!	00000000: 601a 0000 2178 0000 0000 0000 0000 0000  `...!
00000010: 1626 0000 0000 0000 0000 0000 436f 7079  .&...   |	00000010: 1d34 0000 0000 0000 0000 0000 436f 7079  .4...
00000020: 7269 6768 7420 2831 3939 3329 2049 6d61  right	00000020: 7269 6768 7420 2831 3939 3329 2049 6d61  right
00000030: 6769 7465 6320 4465 7369 676e 2c20 496e  gitec	00000030: 6769 7465 6320 4465 7369 676e 2c20 496e  gitec
00000040: 6320 48e7 fffe 23fc 0000 0000 00f1 a114  c H..	00000040: 6320 48e7 fffe 23fc 0000 0000 00f1 a114  c H..
00000050: 23fc 0000 0000 00f1 a100 227c 0000 0000  #....	00000050: 23fc 0000 0000 00f1 a100 227c 0000 0000  #....
00000060: 207c 0000 

In [95]:
#!cat syn6prn

In [118]:
def get_symbol_table(fname):
    syn6 = open(fname,'rb')
    syn6.seek(0)
    magic_number = syn6.read(2)
    text_size = syn6.read(4)
    data_size = syn6.read(4)
    bss_size = syn6.read(4).hex()
    sym_size = syn6.read(4)
    print("Symbol Size",sym_size.hex())
    reserved = syn6.read(10)
    text_data = syn6.read(int.from_bytes(text_size, "big"))

    symbol_table = []
    sym_size_int = int.from_bytes(sym_size, "big")
    for i in range(0,int(sym_size_int/14)):
        symbol = syn6.read(8).decode('utf-8').replace('\x00','')
        info = syn6.read(2).hex()
        sym_address = syn6.read(4).hex()
        symbol_table += [(sym_address,info,symbol)]

    return sorted(symbol_table)

symbol_table = set(get_symbol_table("syn6.o"))
my_symbol_table = get_symbol_table("syn6")


Symbol Size 00001d34
Symbol Size 00001084


In [108]:
set([x[1] for x in symbol_table])

{'8200', '8800', 'a200', 'c000', 'e000'}

In [143]:
print('\n'.join(sorted(set(sorted([f"{x[2]} EQU ${x[0]}" 
                            for x in symbol_table 
                            ])))))


A1_BASE EQU $00f02200
A1_CLIP EQU $00f02208
A1_FINC EQU $00f02220
A1_FLAGS EQU $00f02204
A1_FPIXE EQU $00f02218
A1_FSTEP EQU $00f02214
A1_INC EQU $00f0221c
A1_PIXEL EQU $00f0220c
A1_STEP EQU $00f02210
A2_BASE EQU $00f02224
A2_FLAGS EQU $00f02228
A2_MASK EQU $00f0222c
A2_PIXEL EQU $00f02230
A2_STEP EQU $00f02234
BASE EQU $00f00000
BG EQU $00f00058
BORD1 EQU $00f0002a
BORD2 EQU $00f0002c
B_CMD EQU $00f02238
B_COUNT EQU $00f0223c
B_DSTD EQU $00f02248
B_DSTZ EQU $00f02250
B_I0 EQU $00f02288
B_I1 EQU $00f02284
B_I2 EQU $00f02280
B_I3 EQU $00f0227c
B_IINC EQU $00f02270
B_PATD EQU $00f02268
B_SRCD EQU $00f02240
B_SRCZ1 EQU $00f02258
B_SRCZ2 EQU $00f02260
B_STOP EQU $00f02278
B_Z0 EQU $00f02298
B_Z1 EQU $00f02294
B_Z2 EQU $00f02290
B_Z3 EQU $00f0228c
B_ZINC EQU $00f02274
CHANFLAG EQU $00f1b358
CHANGEFX EQU $000003b0
CHANGE_V EQU $0000042a
CHRO_CLK EQU $00f10014
CLK1 EQU $00f10010
CLK2 EQU $00f10012
CLUT EQU $00f00400
COPYLOOP EQU $00000056
DISABLET EQU $00000000
DISABLE_ EQU $000005a0
DSPORG E

In [140]:
#define	AL_DEFINED	0x8000
#define	AL_EQUATED	0x4000
#define	AL_GLOBAL	0x2000
#define	AL_EQUREG	0x1000
#define	AL_EXTERN	0x0800
#define	AL_DATA		0x0400
#define	AL_TEXT		0x0200
#define	AL_BSS		0x0100
#define	AL_FILE		0x0080

my_symbols = [x[2] for x in my_symbol_table]
print('\n'.join(sorted(set(sorted([f"{x[2]} EQU ${x[0]}" 
                            for x in symbol_table 
                            if x[1]=='e000' and x[2] not in my_symbols])))))


CHANFLAG EQU $00f1b358
DSPORG EQU $00f1b000
FXONLY EQU $00f1b35c
SEMAPHOR EQU $80000080


In [70]:
#TODO: How do we add the relocation table?

## Scratchpad

In [67]:
open("test.s",'w').write("ori.b #$FF,d0")
!./rmac/rmac -s ~oall -n -l*testprn -ps -i test.s -o test
!cat testprn

Legacy mode OFF
    1  00000000  000000FF               ori.b #$FF,d0



In [96]:
open("test.s",'w').write("""
NDI EQU $FF00
.text
lea     $1DE0,a2
byte1: dc.b $00
""")
!./rmac/rmac -s ~oall -n -l*testprn -ps -i test.s -o test
!dd if=test|xxd 2> /dev/null
!cat testprn

Legacy mode OFF
0+1 records in
0+1 records out
68 bytes copied, 2.4071e-05 s, 2.8 MB/s
00000000: 601a 0000 0008 0000 0000 0000 0000 0000  `...............
00000010: 001c 0000 0000 0000 0000 0000 45f9 0000  ............E...
00000020: 1de0 0000 4e44 4900 0000 0000 c000 0000  ....NDI.........
00000030: ff00 6279 7465 3100 0000 8200 0000 0006  ..byte1.........
00000040: 0000 0000                                ....
    1                                   
    2           =0000FF00               NDI EQU $FF00
    3                                   .text
    4  00000000  45F900001DE0           lea     $1DE0,a2
    5  00000006  00                     byte1: dc.b $00

               NDI 000000000000FF00  a 
             byte1 0000000000000006  t 

